In [1]:
import numpy as np
import pandas as pd
import requests
import json

In [2]:
network = "mainnet"

In [3]:
def queryHive(query, network):
    if network == "mainnet":
        url = "https://solitary-wind-b421.hyperfocal-dev.workers.dev/graphql"
    else:
        url = "https://cold-voice-73a0.hyperfocal-dev.workers.dev/graphql"

    response = requests.post(url, query)  
    #print(response.json())
    print(f'status_code: {response.status_code}')

    return response.json()

In [4]:
# Get current position count and block height
# if network == "mainnet":
aperture_manager = "terra1jvehz6d9gk3gl4tldrzd8qzj8zfkurfvtcg99x"


In [5]:
def ret_query(height, address):
    query = """
    {{
      wasm {{
        tag0: contractQuery(
                contractAddress: "{address}"
                query: {{
                  batch_get_position_info: {{
                    positions: [
                      {{
                        chain_id: 3,
                        position_id: "1"
                      }}
                    ]
                  }}
                }}
                height: {height}
              )

      }}

      tendermint {{
        tag0: blockInfo(height: {height}) {{
                block {{
                  header {{
                    height
                    time
                  }}
                }}
              }}
    }}
    }}""".format(address = address, height=height)
    
    return query

def createMasterTable(json_data):
    """
    @todo: need to separate state and collateral range
    """
    columns = pd.MultiIndex.from_tuples([('position','position_id'), ('position', 'chain_id'), ('contract','value'), 
            ('holder','value'), ('position_open_info','height'),('position_open_info','time_nanoseconds'),
            ('position_open_info','uusd_amount'),                   
            ('position_close_info',"value"), ('cdp_idx','value'), ('mirror_asset_cw20_addr','value'),
            ('detailed_info', 'state'),('detailed_info', 'target_collateral_ratio_range'),
            ('detailed_info', 'collateral_ratio'),('detailed_info', 'unclaimed_short_proceeds_uusd_amount'),
            ('detailed_info', 'claimable_short_proceeds_uusd_amount'), ('detailed_info', 'claimable_mir_reward_uusd_value'),
            ('detailed_info', 'claimable_spec_reward_uusd_value'),('detailed_info', 'uusd_value'),
            ('block','height'), ('block','time')])
    
    temp_data = json_data['data']['wasm']['tag0']['items'][0]
    temp_info = json_data['data']['wasm']['tag0']['items'][0]['info']
    temp_detail = json_data['data']['wasm']['tag0']['items'][0]['info']['detailed_info']
    tendetmint = json_data['data']['tendermint']['tag0']['block']['header']
    out = pd.DataFrame(columns = columns, data = [[temp_data['position']['position_id'],
                    temp_data['position']['chain_id'], temp_data['contract'],
                    temp_data['holder'], temp_info['position_open_info']['height'],
                    temp_info['position_open_info']['time_nanoseconds'],temp_info['position_open_info']['uusd_amount'],
                    temp_info['position_close_info'],
                    temp_info['cdp_idx'], temp_info['mirror_asset_cw20_addr'],temp_detail['state'],
                    temp_detail['target_collateral_ratio_range'],temp_detail['collateral_ratio'],
                    temp_detail['unclaimed_short_proceeds_uusd_amount'],temp_detail['claimable_short_proceeds_uusd_amount'],
                    temp_detail['claimable_mir_reward_uusd_value'],temp_detail['claimable_spec_reward_uusd_value'],
                    temp_detail['uusd_value'], tendetmint['height'], tendetmint['time']]]) 
    
    
    return out

In [6]:
address = "terra1jvehz6d9gk3gl4tldrzd8qzj8zfkurfvtcg99x"
height_start = 7000000 # around Feb (3 months back as of 20220510)
height_end = 7560000 
step = 1
output = None
count = 0
for height in range(height_start, height_end, step):
    json_data = queryHive({"query": ret_query(height, address)}, network)
    temp = createMasterTable(json_data)
#     print(height)
    print(json_data)
    if output is None:
        output = temp
    else:
        output = output.append(temp)
    count+=1
    if(count==10):
        break

status_code: 200
{'data': {'wasm': {'tag0': {'items': [{'position': {'chain_id': 3, 'position_id': '1'}, 'contract': 'terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da', 'holder': 'terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9', 'info': {'position_open_info': {'height': 6298808, 'time_nanoseconds': 1643659682234717400, 'uusd_amount': '1000000000'}, 'position_close_info': None, 'cdp_idx': '78099', 'mirror_asset_cw20_addr': 'terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2', 'detailed_info': {'cdp_preemptively_closed': False, 'state': {'uusd_balance': '3025234', 'uusd_long_farm': '338449382', 'mirror_asset_short_amount': '1949928', 'mirror_asset_balance': '101748', 'mirror_asset_long_farm': '1901343', 'mirror_asset_long_amount': '2003091', 'collateral_anchor_ust_amount': '566483023', 'collateral_uusd_value': '705661656', 'mirror_asset_oracle_price': '178.44', 'anchor_ust_oracle_price': '1.245688974953820579', 'terraswap_pool_info': {'lp_token_amount': '24897810', 'lp_token_cw20_addr': 'terra1yjvlg

/var/folders/fb/xfm534l94b163z6454wbkn_w0000gn/T/ipykernel_75376/578923498.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(temp)


status_code: 200
{'data': {'wasm': {'tag0': {'items': [{'position': {'chain_id': 3, 'position_id': '1'}, 'contract': 'terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da', 'holder': 'terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9', 'info': {'position_open_info': {'height': 6298808, 'time_nanoseconds': 1643659682234717400, 'uusd_amount': '1000000000'}, 'position_close_info': None, 'cdp_idx': '78099', 'mirror_asset_cw20_addr': 'terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2', 'detailed_info': {'cdp_preemptively_closed': False, 'state': {'uusd_balance': '3025234', 'uusd_long_farm': '338449382', 'mirror_asset_short_amount': '1949928', 'mirror_asset_balance': '101748', 'mirror_asset_long_farm': '1901343', 'mirror_asset_long_amount': '2003091', 'collateral_anchor_ust_amount': '566483023', 'collateral_uusd_value': '705661657', 'mirror_asset_oracle_price': '178.44', 'anchor_ust_oracle_price': '1.245688976559452361', 'terraswap_pool_info': {'lp_token_amount': '24897810', 'lp_token_cw20_addr': 'terra1yjvlg

/var/folders/fb/xfm534l94b163z6454wbkn_w0000gn/T/ipykernel_75376/578923498.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(temp)


status_code: 200
{'data': {'wasm': {'tag0': {'items': [{'position': {'chain_id': 3, 'position_id': '1'}, 'contract': 'terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da', 'holder': 'terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9', 'info': {'position_open_info': {'height': 6298808, 'time_nanoseconds': 1643659682234717400, 'uusd_amount': '1000000000'}, 'position_close_info': None, 'cdp_idx': '78099', 'mirror_asset_cw20_addr': 'terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2', 'detailed_info': {'cdp_preemptively_closed': False, 'state': {'uusd_balance': '3025234', 'uusd_long_farm': '338449382', 'mirror_asset_short_amount': '1949928', 'mirror_asset_balance': '101748', 'mirror_asset_long_farm': '1901343', 'mirror_asset_long_amount': '2003091', 'collateral_anchor_ust_amount': '566483023', 'collateral_uusd_value': '705661657', 'mirror_asset_oracle_price': '178.44', 'anchor_ust_oracle_price': '1.245688976559452361', 'terraswap_pool_info': {'lp_token_amount': '24897810', 'lp_token_cw20_addr': 'terra1yjvlg

...

/var/folders/fb/xfm534l94b163z6454wbkn_w0000gn/T/ipykernel_75376/578923498.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(temp)


status_code: 200
{'data': {'wasm': {'tag0': {'items': [{'position': {'chain_id': 3, 'position_id': '1'}, 'contract': 'terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da', 'holder': 'terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9', 'info': {'position_open_info': {'height': 6298808, 'time_nanoseconds': 1643659682234717400, 'uusd_amount': '1000000000'}, 'position_close_info': None, 'cdp_idx': '78099', 'mirror_asset_cw20_addr': 'terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2', 'detailed_info': {'cdp_preemptively_closed': False, 'state': {'uusd_balance': '3025234', 'uusd_long_farm': '338449382', 'mirror_asset_short_amount': '1949928', 'mirror_asset_balance': '101748', 'mirror_asset_long_farm': '1901343', 'mirror_asset_long_amount': '2003091', 'collateral_anchor_ust_amount': '566483023', 'collateral_uusd_value': '705661656', 'mirror_asset_oracle_price': '178.44', 'anchor_ust_oracle_price': '1.245688975978018095', 'terraswap_pool_info': {'lp_token_amount': '24897810', 'lp_token_cw20_addr': 'terra1yjvlg

/var/folders/fb/xfm534l94b163z6454wbkn_w0000gn/T/ipykernel_75376/578923498.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(temp)


status_code: 200
{'data': {'wasm': {'tag0': {'items': [{'position': {'chain_id': 3, 'position_id': '1'}, 'contract': 'terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da', 'holder': 'terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9', 'info': {'position_open_info': {'height': 6298808, 'time_nanoseconds': 1643659682234717400, 'uusd_amount': '1000000000'}, 'position_close_info': None, 'cdp_idx': '78099', 'mirror_asset_cw20_addr': 'terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2', 'detailed_info': {'cdp_preemptively_closed': False, 'state': {'uusd_balance': '3025234', 'uusd_long_farm': '338449382', 'mirror_asset_short_amount': '1949928', 'mirror_asset_balance': '101748', 'mirror_asset_long_farm': '1901343', 'mirror_asset_long_amount': '2003091', 'collateral_anchor_ust_amount': '566483023', 'collateral_uusd_value': '705661656', 'mirror_asset_oracle_price': '178.44', 'anchor_ust_oracle_price': '1.245688975723159172', 'terraswap_pool_info': {'lp_token_amount': '24897810', 'lp_token_cw20_addr': 'terra1yjvlg

/var/folders/fb/xfm534l94b163z6454wbkn_w0000gn/T/ipykernel_75376/578923498.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(temp)


In [7]:
output

position                                               contract  \
  position_id chain_id                                         value   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   
0           1        3  terra1e83dv235jzz4mf273hwscx7shhw7qp35l9w0da   

                                         holder position_open_info  \
                                          value             height   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   
0  terra1hqew4vjl8k0dx9rm7lrcj75j9w29qmun6tk3s9            6298808   

                                   position_close_info cdp_idx  \
      time_nanoseconds uusd_amount               value   value   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   
0  1643659682234717400  1000000000                None   78099   

                         mirror_asset_cw20_addr  \
                                          value   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   
0  terra1ptdxmj3xmmljzx02nr4auwfuelmj0cnkh8egs2   

                                       detailed_info  \
                                               state   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   
0  {'uusd_balance': '3025234', 'uusd_long_farm': ...   

                                                       \
  target_collateral_ratio_range      collateral_ratio   
0  {'min': '1.8', 'max': '2.2'}  2.028083023843941932   
0  {'min': '1.8', 'max': '2.2'}  2.02808302671795

In [141]:
#things to do
#1. It seems data are not chaning by iterating block height, we might need to iterate block height after some step

dict_keys(['state', 'target_collateral_ratio_range', 'collateral_ratio', 'unclaimed_short_proceeds_uusd_amount', 'claimable_short_proceeds_uusd_amount', 'claimable_mir_reward_uusd_value', 'claimable_spec_reward_uusd_value', 'uusd_value'])

In [ ]:
# to do: what each the columns are 
# block heights -> query data frequency
# create github